In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create network model
model = Network('model')

# Block 1: Create nodes
node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PSPRIO)  # Preemptive Static Priority

In [3]:
# Block 2: Create job classes with different priorities
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 2, node[0], 0)  # 2 jobs, priority 0 (low)
jobclass[1] = ClosedClass(model, 'Class2', 2, node[0], 1)  # 2 jobs, priority 1 (high)

In [4]:
# Block 3: Set service time distributions - following MATLAB specification exactly
# Delay station (think time)
node[0].set_service(jobclass[0], Erlang(3, 2))  # Erlang(k=3, rate=2)
node[0].set_service(jobclass[1], HyperExp(0.5, 3.0, 10.0))  # HyperExp(p=0.5, rate1=3.0, rate2=10.0)

# PSPRIO Queue station
node[1].set_service(jobclass[0], HyperExp(0.1, 1.0, 10.0))  # HyperExp(p=0.1, rate1=1.0, rate2=10.0)
node[1].set_service(jobclass[1], Exp(1))  # Exponential with rate=1

In [5]:
# Block 4: Set routing (serial routing for both classes) - following MATLAB specification exactly
P = model.init_routing_matrix()
# P{1} = Network.serial_routing(node) equivalent for jobclass[0]
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)  # Delay -> Queue1
P.set(jobclass[0], jobclass[0], node[1], node[0], 1.0)  # Queue1 -> Delay
# P{2} = Network.serial_routing(node) equivalent for jobclass[1]
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)  # Delay -> Queue1
P.set(jobclass[1], jobclass[1], node[1], node[0], 1.0)  # Queue1 -> Delay
model.link(P)

In [6]:
# Model information
M = model.get_number_of_stations()
K = model.get_number_of_classes()

In [7]:
# CTMC removed - not tested in JAR for prio_psprio
# solver_ctmc = CTMC(model)
# avg_table_ctmc = solver_ctmc.avg_table()

In [8]:
# Solve with JMT simulation - skip due to PSPRIO JMT support issue
print("\\nSOLVER: JMT (skipped - PSPRIO scheduling not supported in JMT)")
print("JMT cannot handle PSPRIO scheduling strategy - ClassNotFoundException for EPSStrategyPriority")
# solver_jmt = JMT(model, seed=23000, verbose=VerboseLevel.STD, samples=5000)
# avg_table_jmt = solver_jmt.avg_table()

\nSOLVER: JMT (skipped - PSPRIO scheduling not supported in JMT)
JMT cannot handle PSPRIO scheduling strategy - ClassNotFoundException for EPSStrategyPriority


In [9]:
# SSA with seed=23000, samples=5000, verbose=SILENT to match JAR test
solver_ssa = SSA(model, seed=23000, samples=5000, verbose=VerboseLevel.SILENT)
avg_table_ssa = solver_ssa.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 1.0170 1.0170 0.6421  0.6421 1.5838 1.5838
  Delay   Class2 0.1405 0.1405 0.2120  0.2120 0.6630 0.6630
 Queue1   Class1 0.9830 0.3000 0.6207  0.6207 1.5838 1.5838
 Queue1   Class2 1.8595 0.6958 2.8124  2.8124 0.6612 0.6612
